## モデル読み込みと関数定義

In [1]:
import whisper 
model = whisper.load_model("large")

/home/kbylab/ogis2024/venv3_12_1/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location

#### 各セグメントのembeddingを返す

In [2]:
import numpy as np
import contextlib
import wave
#from pyannote.audio import Pipeline
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment, notebook

def segment_embedding(
    file_name: str,
    duration: float,
    segment,
    embedding_model: PretrainedSpeakerEmbedding
) -> np.ndarray:
    """
    音声ファイルから指定されたセグメントの埋め込みを計算します。
    
    Parameters
    ----------
    file_name: str
        音声ファイルのパス
    duration: float
        音声ファイルの継続時間
    segment: whisperのtranscribeのsegment
    embedding_model: PretrainedSpeakerEmbedding
        埋め込みモデル

    Returns
    -------
    np.ndarray
        計算された埋め込みベクトル
    """
    audio = Audio()
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(file_name, clip)
    #print(waveform)
    return embedding_model(waveform[None])

def generate_speaker_embeddings(
    meeting_file_path: str,
    transcript
) -> np.ndarray:
    """
    音声ファイルから話者の埋め込みを計算します。
    
    Parameters
    ----------
    meeting_file_path: str
        音声ファイルのパス
    transcript: Whisper API の transcribe メソッドの出力結果

    Returns
    
    -------
    np.ndarray
        計算された話者の埋め込み群
    """
    segments = transcript['segments']
    embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device='cuda')
    embeddings = np.zeros(shape=(len(segments), 192))

    with contextlib.closing(wave.open(meeting_file_path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)

    for i, segment in enumerate(segments):
        embeddings[i] = segment_embedding(meeting_file_path, duration, segment, embedding_model)

    embeddings = np.nan_to_num(embeddings)
    return embeddings


/home/kbylab/ogis2024/venv3_12_1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 埋め込みから各セグをクラスタリング(照合するなら無視でok)

In [3]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from typing import List, Tuple

def clustering_embeddings(speaker_count: int, embeddings: np.ndarray) -> AgglomerativeClustering:
    """
    埋め込みデータをクラスタリングして、クラスタリングオブジェクトを返します。

    Parameters
    ----------
    embeddings: np.ndarray
        分散表現（埋め込み）のリスト。

    Returns
    -------
    AgglomerativeClustering
        クラスタリングオブジェクト。
    """
    clustering = AgglomerativeClustering(speaker_count).fit(embeddings)
    return clustering

def format_speaker_output_by_segment(clustering: AgglomerativeClustering, transcript: dict) -> str:
    """
    クラスタリングの結果をもとに、各発話者ごとにセグメントを整形して出力します

    Parameters
    ----------
    clustering: AgglomerativeClustering
        クラスタリングオブジェクト。
    transcript: dict
        Whisper API の transcribe メソッドの出力結果

    Returns
    -------
    str
        発話者ごとに整形されたセグメントの文字列
    """
    labeled_segments = []
    for label, segment in zip(clustering.labels_, transcript["segments"]):
        labeled_segments.append((label, segment["start"], segment["text"]))

    output = ""
    for speaker, _, text in labeled_segments:
        output += f"話者{speaker + 1}: 「{text}」\n"
    return output


#### 各embeddingごとにサンプル音声とcos類似度で照合

In [4]:
from scipy.spatial.distance import cosine
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from typing import List, Tuple

def closest_reference_speaker(embedding: np.ndarray, references: List[Tuple[str, np.ndarray]]) -> str:
    """
    与えられた埋め込みに最も近い参照話者を返します。

    Parameters
    ----------
    embedding: np.ndarray
        話者の埋め込み
    references: List[Tuple[str, np.ndarray]]
        参照話者の名前と埋め込みのリスト

    Returns
    -------
    str
        最も近い参照話者の名前
    """
    min_distance = float('inf')
    closest_speaker = None
    for name, reference_embedding in references:
        #print(reference_embedding.shape)
        reference_embedding = reference_embedding[0]
        distance = cosine(embedding, reference_embedding)
        if distance < min_distance:
            min_distance = distance
            closest_speaker = name

    return closest_speaker

def format_speaker_output_by_segment2(embeddings: np.ndarray, transcript: dict, reference_embeddings: List[Tuple[str, np.ndarray]]) -> str:
    """
    各発話者の埋め込みに基づいて、セグメントを整形して出力します。

    Parameters
    ----------
    embeddings: np.ndarray
        話者の埋め込みのリスト
    transcript: dict
        Whisper API の transcribe メソッドの出力結果
    reference_embeddings: List[Tuple[str, np.ndarray]]
        参照話者の名前と埋め込みのリスト

    Returns
    -------
    str
        発話者ごとに整形されたセグメントの文字列。
    """
    labeled_segments = []
    for embedding, segment in zip(embeddings, transcript["segments"]):
        #print(embedding.shape)
        speaker_name = closest_reference_speaker(embedding, reference_embeddings)
        labeled_segments.append((speaker_name, segment["start"], segment["text"]))

    output = ""
    for speaker, _, text in labeled_segments:
        output += f"{speaker}: 「{text}」\n"
    return output


## 文字起こしと話者識別

#### クラスタリングver

In [6]:
#文字起こしする音声
FILE_PATH = "../resource/record241023/meeting_take2_mono.wav"
#FILE_PATH = "../resource/meeting_voice/meeting_demo_special_record_deluxe.wav"
#FILE_PATH = "../resource/onsei_kaigi1015/OGISmeeting.wav"
#FILE_PATH = "../resource/output.wav"
num_people = 3  #話者の数


res_meet = model.transcribe(FILE_PATH, verbose=False, language="ja")
#print(res_meet)
embed_meet = generate_speaker_embeddings(FILE_PATH,res_meet)

clus_embed_meet = clustering_embeddings(num_people,embed_meet)
format_out_meet = format_speaker_output_by_segment(clus_embed_meet, res_meet)
print(format_out_meet)


100%|██████████| 6911/6911 [00:17<00:00, 391.89frames/s]
/home/kbylab/ogis2024/venv3_12_1/lib/python3.12/site-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to

話者1: 「ではこれから新製品のマーケティング戦略に関する」
話者1: 「社内会議を始めます」
話者1: 「まず新製品のターゲット層についてなんですが」
話者1: 「30代から40代のビジネスマンに絞りましょう」
話者1: 「高機能でサイリッシュなデザインが売りになります」
話者1: 「でも最近の若い人たちも」
話者1: 「同じ製品を求めている傾向があると」
話者1: 「現場の声から聞いています」
話者1: 「それに価格帯ももう少し広げた方が」
話者1: 「いや30代以上に限定すべきだと思います」
話者1: 「若い層はどうせ似たようなものを買うでしょう」
話者2: 「私たちが開発した技術は」
話者2: 「幅広い年齢層にアピールできるものです」
話者2: 「技術的にも他の層に響く可能性が」
話者1: 「でもリソースが限られているんです」
話者1: 「今のマーケティング戦略を変更する余裕はありません」
話者1: 「それは理解していますが」
話者1: 「もし市場の動向が変わったら」
話者1: 「最初からシェアを狭めるのはリスクだと思います」
話者1: 「うーんそこまで考える時間はないんです」
話者1: 「今はこの路線で行きます」
話者3: 「はい」
話者2: 「わかりました」
話者2: 「しかしもし失敗したらその時の責任は」
話者1: 「失敗しません」
話者1: 「私たちはこれで行きます」
話者1: 「はい以上です」



#### 照合ver

In [7]:
#文字起こしする音声
FILE_PATH = "../resource/record241023/meeting_take2_mono.wav"
# FILE_PATH = "../resource/meeting_voice/meeting_demo_special_record_deluxe.wav"

#照合用個人の音声
FILE_PATH_DICT = {"hanagata": "../resource/record241023/sample_hanagata.wav", 
                  "isizawa": "../resource/record241023/sample_ishizawa.wav",
                  "shibasaki": "../resource/record241023/sample_shibasaki.wav"}
# FILE_PATH_DICT = {"hanagata": "../resource/onsei_kaigi1015/hanagata_sample_voice.wav", 
#                   "isizawa": "../resource/meeting_voice/ishizawa_sample.wav",
#                   "shibasaki": "../resource/onsei_kaigi1015/shibasaki_sample.wav"}


ref = []
res_meet = model.transcribe(FILE_PATH, verbose=False, language="ja")
embed_meet = generate_speaker_embeddings(FILE_PATH,res_meet)

for name, file in FILE_PATH_DICT.items():
    res = model.transcribe(file, verbose=False, language="ja")
    embed = generate_speaker_embeddings(file,res)
    #clus_embed = clustering_embeddings(num_people, embed)
    #format_out = format_speaker_output_by_segment(clus_embed, res)
    ref.append((name,embed))
close_ref_speaker = format_speaker_output_by_segment2(embed_meet,res_meet,ref)

print(close_ref_speaker)




100%|██████████| 6911/6911 [00:17<00:00, 405.49frames/s]
/home/kbylab/ogis2024/venv3_12_1/lib/python3.12/site-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to

hanagata: 「ではこれから新製品のマーケティング戦略に関する」
hanagata: 「社内会議を始めます」
hanagata: 「まず新製品のターゲット層についてなんですが」
hanagata: 「30代から40代のビジネスマンに絞りましょう」
hanagata: 「高機能でサイリッシュなデザインが売りになります」
isizawa: 「でも最近の若い人たちも」
isizawa: 「同じ製品を求めている傾向があると」
isizawa: 「現場の声から聞いています」
isizawa: 「それに価格帯ももう少し広げた方が」
hanagata: 「いや30代以上に限定すべきだと思います」
hanagata: 「若い層はどうせ似たようなものを買うでしょう」
shibasaki: 「私たちが開発した技術は」
shibasaki: 「幅広い年齢層にアピールできるものです」
shibasaki: 「技術的にも他の層に響く可能性が」
hanagata: 「でもリソースが限られているんです」
hanagata: 「今のマーケティング戦略を変更する余裕はありません」
isizawa: 「それは理解していますが」
hanagata: 「もし市場の動向が変わったら」
isizawa: 「最初からシェアを狭めるのはリスクだと思います」
hanagata: 「うーんそこまで考える時間はないんです」
hanagata: 「今はこの路線で行きます」
hanagata: 「はい」
hanagata: 「わかりました」
shibasaki: 「しかしもし失敗したらその時の責任は」
hanagata: 「失敗しません」
hanagata: 「私たちはこれで行きます」
hanagata: 「はい以上です」



### テキストに保存

In [5]:
print(close_ref_speaker)
with open("../resource/out_puts/test.txt", 'w', encoding="utf-8") as file:
    file.write(close_ref_speaker)


hanagata: 「今日これから新製品についての会議を始めます。」
hanagata: 「ターゲット層についてなんですが、」
isizawa: 「新製品のターゲット層は30代から40代のビジネスマンに絞りましょう。」
isizawa: 「高企業でスタイリッシュのデザインが有利になると思います。」
isizawa: 「でも最近の若い人たちも同じ製品を求めている傾向があると現場の声が聞こえます。」
isizawa: 「それに価格単位をもう少し広げた方が、」
isizawa: 「いや、30代以上に限定するべきだと思います。」
isizawa: 「若い層がどうせ似たようなものを買うでしょう。」
shibasaki: 「私たちが開発した技術は幅広いデザイン層でうっきりできるものです。」
shibasaki: 「技術的にも他の層に響く可能性があります。」
isizawa: 「でも、2層しか限られているんです。」
isizawa: 「今のパワーリーティング戦でこれでもこうする余裕はありません。」
isizawa: 「それは理解していますが、」
hanagata: 「もし市場の動向が変わったら、」
isizawa: 「最初から知恵を狭めるようなリスクがあると思います。」
hanagata: 「うーん、」
hanagata: 「考える時間はないんです。」
isizawa: 「今はこの路線で行きます。」
shibasaki: 「わかりました。」
shibasaki: 「しかし、もし失敗したらその時の責任は?」
isizawa: 「失敗しません。」
isizawa: 「私たちはこれで行きます。」
isizawa: 「今日の会議は終わりです。」



In [ ]:
#####csv ver
speech_data = pd.DataFrame(res["segments"])[["start", "end", "text"]]
pd.set_option("display.max_rows", len(speech_data))
speech_data